<a href="https://colab.research.google.com/github/vincentdavis/zwift_maps/blob/main/zwift_multi_id_event_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use this to map an event recorded at dashboard.vyrtyfy.com with multiple riders**

In [2]:
!rm -r zwift_maps
!git clone https://github.com/vincentdavis/zwift_maps.git

rm: cannot remove 'zwift_maps': No such file or directory
Cloning into 'zwift_maps'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 4), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [48]:
from datetime import datetime
import json
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.io as pio

def world_time_to_timestamp(world_time):
    return datetime.fromtimestamp((int(world_time) + 1414016075000) / 1000.0)

def multiple_id_event(json_path, event_start_time):
  """
  You need to look up the event start time in GMT, UTC.
  This start tiem is used to filter the data. IT is possible the rider,
  never start the event or quits and rider somewhere else, we need to filter these.
  """
  with open(json_path) as j:
    data = json.load(j)
  row_list = []
  for id, id_data in data.items():
    if id_data is not None:
          for row in id_data.values():
            row.update({'id':id})
            row_list.append(row)
  df = pd.DataFrame(row_list)
  df['worldTime'] = df.worldTime.astype(int)
  df['irlTime'] = pd.to_datetime(df['worldTime'], unit='ms') + pd.Timedelta(seconds = (1414016075))
  start_time = datetime.strptime('2021-01-21 19:40', '%Y-%m-%d %H:%M')
  # event_start_time = np.datetime64(start_time)

  # Filters
  df = df[df.irlTime >= event_start_time]
  df = df[df.speed.notnull()]

  df.sort_values(by=['irlTime'], inplace=True)
  df = df.reset_index(drop=True)
  df['-y'] = -df.y
  return df


In [49]:
# Tick Tock course:
Tick_Tock_course = 'zwift_maps/event_records/Tick_Tock-2laps-event-1550562.json'
Tick_Tock_course_start_time = datetime.strptime('2021-01-21 19:40', '%Y-%m-%d %H:%M')

df = multiple_id_event(Tick_Tock_course, Tick_Tock_course_start_time)
print(df.columns)
df = df[['id', 'distance', 'heading', 'heartrate', 'power', 'progress', 'roadTime', 'speed', 'time', 'timestamp', 'worldTime', 'irlTime', 'x', 'y', 'groupId', 'laps', 'altitude']]

Index(['altitude', 'cadenceUHz', 'calories', 'climbing', 'distance', 'f19',
       'f20', 'heading', 'heartrate', 'id', 'irlTime', 'lean', 'power',
       'progress', 'roadPosition', 'roadTime', 'speed', 'time', 'timestamp',
       'watchingRiderId', 'worldTime', 'x', 'y', 'customisationId', 'groupId',
       'justWatching', 'laps', '-y'],
      dtype='object')


In [83]:
# more filters
df = multiple_id_event(Tick_Tock_course, Tick_Tock_course_start_time)
df = df[df.x > -200000]
df = df[df.y > 40500]
df = df[df.groupId.notnull()]
# grouped = df.groupby('id')
# df = grouped.filter(lambda x: x['time'].max() >=2400)
fig = px.scatter(df, x="x", y="y",
                color="roadTime", color_continuous_scale=px.colors.sequential.Viridis
                )
fig.show()

In [85]:
df = df[['x','y','roadTime', 'heading', 'altitude']]

In [104]:
min_max = {'x_max': df.x.max(), 
          'x_min': df.x.min(),
          'y_max' : df.x.max(),
          'y_min' : df.x.min(),
          'roadTime_max' : df.x.max(),
          'roadTime_min' : df.x.min(),
          'altitude_max' : df.x.max(),
          'altitude_min' : df.x.min()}
print(min_max)

{'x_max': 130139.57, 'x_min': -192691.05, 'y_max': 130139.57, 'y_min': -192691.05, 'roadTime_max': 130139.57, 'roadTime_min': -192691.05, 'altitude_max': 130139.57, 'altitude_min': -192691.05}


In [102]:
  df.sort_values(by=['roadTime'], inplace=True)

  fig = px.scatter(df, x="x", y="y",
                color="roadTime", color_continuous_scale=px.colors.sequential.Viridis
                )
fig.show()

In [107]:
from google.colab import files
df.to_csv('Tick_Tock_race_course')
files.download('Tick_Tock_race_course')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>